In [2]:
import datalab.storage as storage
import pandas as pd
from io import BytesIO
import urllib.request
import numpy as np

In [3]:
bucket = storage.Bucket('cs221-flight-data')
flights_data = bucket.item('flights.csv')
uri = flights_data.uri
%gcs read --object $uri --variable data
data = pd.read_csv(BytesIO(data))

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
relevant_cols = data[['ARRIVAL_DELAY', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'TAIL_NUMBER']]
relevant_cols = relevant_cols[relevant_cols['ORIGIN_AIRPORT'] == 'ORD']
relevant_cols = relevant_cols[relevant_cols['AIRLINE'].isin(['MQ', 'UA', 'AA', 'EV', 'OO'])]
for row in range(relevant_cols.shape[0]):
    if row % 10000 == 0:
        print ('done ' + str(row))
    delay = relevant_cols.iloc[row, 0]
    if delay < 0:
        relevant_cols.iloc[row, 0] = 0
    elif delay <= 15:
        relevant_cols.iloc[row, 0] = 1
    else:
        relevant_cols.iloc[row, 0] = 2
    # bucket the departure by hour
    relevant_cols.iloc[row, 6] = int(relevant_cols.iloc[row, 6]) // 100

done 0
done 10000
done 20000
done 30000
done 40000
done 50000
done 60000
done 70000
done 80000
done 90000
done 100000
done 110000
done 120000
done 130000
done 140000
done 150000
done 160000
done 170000
done 180000
done 190000
done 200000
done 210000
done 220000
done 230000
done 240000
done 250000
done 260000
done 270000
done 280000


In [12]:
bucket.item('flights-limited.csv').write_to(relevant_cols.to_csv(),'text/csv')

In [9]:
# get the flight information
years = []
models = []
tail_numbers = data['TAIL_NUMBER'].values.astype(str)
tail_numbers = np.unique(tail_numbers)
print(len(tail_numbers))
valid_tail_nums = []
for tail_num in tail_numbers:
    url = 'http://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt={0}'.format(tail_num)
    with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')
        if html.find("<h5>Can Not Process Your Request At This Time</h5>") == -1:
           # then there wasn't an error
            start_index = html.find('<span id="content_Label7" class="Results_DataText">')
            length = len('<span id="content_Label7" class="Results_DataText">')
            end_index = html.find("</span>", start_index)
            model = html[start_index+length:end_index]
            start_index = html.find('<span id="content_Label17" class="Results_DataText">')
            length = len('<span id="content_Label17" class="Results_DataText">')
            end_index = html.find("</span>", start_index)
            year = html[start_index+length:end_index]
            if len(year) > 0 and len(model) > 0:
                valid_tail_nums.append(tail_num)
                years.append(year)
                models.append(model)

4898


In [10]:
print(len(valid_tail_nums))

4405


In [11]:
# create a new dataframe with all of the airplane features
airplane_data = {'TAIL_NUMBER':valid_tail_nums, 'YEAR':years, 'MODEL':models}
airplane_df = pd.DataFrame(data=airplane_data)
airplane_df.to_csv("airplane_info.csv")
bucket.item('airplane.csv').write_to(airplane_df.to_csv(),'text/csv')

0
